# LIMPIEZA DEL DATAFRAME DE METARS

## En este Notebook se va a realizar la limpieza de los partes METAR obtenidos anteriormente. El resultado final de la limpieza es un DataFrame con las siguientes columnas:

- **Metar_id**: Columna con un id único para cada parte Metar, para poder relacionarlo posteriormente con los vuelos.
- **Date_time**: Columna en formato *Datetime* con la fecha y la hora de emisión del parte.
- **Day**: Columna con el día en el que se emitió el parte en formato YYYY-MM-DD.
- **Hour**: Hora en la que se emitió el parte en formato HH:MM.
- **Condition**: Condición meteorológica del parte.
- **Temperature**: Temperatura en grados Celsius [º].
- **Wind**: Velocidad del viento en nudos o millas naúticas por hora [knots].
- **Gusts**: Velocidad de ráfagas si las hubiere en nudos o millas naúticas por hora [knots].
- **Relative_hum**: Humedad relativa en tanto por ciento [%].
- **Pressure**: Presión atmosférica en hectopascales [hPa].

In [8]:
import time
import numpy as np
import pylab as plt   
import seaborn as sns
import pandas as pd
import re
import sys
sys.path.append('../src')
from funmetar import *

## Se cargan y concatenan los datos de los meses de interés en el orden deseado

In [9]:
met_oct_23 = pd.read_csv("../metar/metar_october_2023.csv")
met_sep_23 = pd.read_csv("../metar/metar_september_2023.csv")
met_aug_23 = pd.read_csv("../metar/metar_august_2023.csv")
met_jul_23 = pd.read_csv("../metar/metar_july_2023.csv")
met_jun_23 = pd.read_csv("../metar/metar_june_2023.csv")
met_may_23 = pd.read_csv("../metar/metar_may_2023.csv")
met_apr_23 = pd.read_csv("../metar/metar_april_2023.csv")
met_mar_23 = pd.read_csv("../metar/metar_march_2023.csv")
met_feb_23 = pd.read_csv("../metar/metar_february_2023.csv")
met_jan_23 = pd.read_csv("../metar/metar_january_2023.csv")
met_dec_22 = pd.read_csv("../metar/metar_december_2022.csv")
met_nov_22 = pd.read_csv("../metar/metar_november_2022.csv")


In [10]:
mt = pd.concat([met_oct_23,met_sep_23,met_aug_23,met_jul_23,met_jun_23,met_may_23,
               met_apr_23,met_mar_23,met_feb_23,met_jan_23,met_dec_22,met_nov_22])

In [11]:
mt.head()

,Day,Hour,Condition,Temperature,Wind,Relative_hum,Pressure
0,Sunday 1 October 2023,00:00,Clear,20°,4,73%,1025 hPa
1,Sunday 1 October 2023,00:30,Clear,19°,6,78%,1025 hPa
2,Sunday 1 October 2023,01:00,Clear,19°,6,78%,1025 hPa
3,Sunday 1 October 2023,01:30,Clear,19°,6,73%,1025 hPa
4,Sunday 1 October 2023,02:00,Clear,18°,6,77%,1025 hPa


In [12]:
mt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17722 entries, 0 to 1463
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Day           17722 non-null  object
 1   Hour          17722 non-null  object
 2   Condition     17722 non-null  object
 3   Temperature   17722 non-null  object
 4   Wind          17722 non-null  object
 5   Relative_hum  17722 non-null  object
 6   Pressure      17722 non-null  object
dtypes: object(7)
memory usage: 1.1+ MB


In [13]:
mt.shape

(17722, 7)

## Los valores de las ráfagas (cuando las hay) se encuentran desafortunadamente en la columna de "Pressure" tras la estracción.
## Por ello se crea una función (<span style="color:blue">generar_columna_gust</span>) que extraiga los valores en una nueva columna llamada "Gusts".

In [14]:
mt['gusts'] = mt['Pressure'].apply(generar_columna_gust)

In [15]:
help(generar_columna_gust)

Help on function generar_columna_gust in module funmetar:

generar_columna_gust(valor)
    Extrae los dos primeros dígitos seguidos de un espacio del valor de la columna pressure
    ya que erroneamente los valores de ráfagas(gusts) cuando los hay se encuentran es esta columna.
    
    Parameters
    ----------
    valor : str
        Cadena de entrada desde la cual se intentan extraer los dos primeros dígitos.
    
    Returns
    -------
    str or int
        Si se encuentran dos dígitos seguidos de un espacio al principio de 'valor', se devuelve 
        la cadena que representa esos dos dígitos. De lo contrario, se devuelve 0.



## La columna *Relative_hum* viene algo sucia tras la extracción ya que cuando en el registro hay ráfagas, en la columna aparece el símbolo ">" y el valor de la humedad ha quedado desplazado a la siguiente columna.

## Por ello, se crea una función (<span style="color:blue">fix_hum_column</span>) que corrija la columna cuando sea necesario.

In [16]:
mt['Relative_hum'] = mt.apply(fix_hum_column, axis=1)


## En la columna *Pressure* nos quedamos únicamente con el valor numérico de la presión.

In [17]:
mt['Pressure'] = mt['Pressure'].apply(lambda x: int(x[-8:-4]))

## En la columna *Wind*, en ocasiones viene el dato numérico de la velocidad del viento y en otros caso el string *"Calm"*. Corregimos la columna para que en esos casos ponga *"0"*.

In [18]:
mt['Wind'] = mt['Wind'].apply(lambda x: 0 if x=="Calm" else x)

## Hacemos un cambio de variable a nudos en las columnas *"Wind"* y *"gusts"*.

In [19]:
conversion = 1.852
mt['Wind'] = mt['Wind'].apply(lambda x: int(round(float(x)/conversion,0)))
mt['gusts'] = mt['gusts'].apply(lambda x: int(round(float(x)/conversion,0)))

## En la columna *Temperature* nos quedamos únicamente con el valor numérico de la temperatura.

In [20]:
mt['Temperature'] = mt['Temperature'].apply(lambda x: int(x[:-1]))

## Hay aún 5 registros (de más de 17000) en los que aún aparece el símbolo ">" en la columna *Relative_hum* .  Se decide reemplazar el valor de estos registros por la moda de la columna.

In [21]:
mt["Relative_hum"][mt.Relative_hum == ">"] = mt['Relative_hum'].mode().iloc[0]

C:\Users\daarr\AppData\Local\Temp\ipykernel_97920\3997667125.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mt["Relative_hum"][mt.Relative_hum == ">"] = mt['Relative_hum'].mode().iloc[0]


## En la columna *Relative_hum* nos quedamos únicamente con el valor numérico( en tanto por ciento) de la humedad.

In [22]:
mt['Relative_hum'] = mt['Relative_hum'].apply(lambda x: int(x[:-1]) if x.endswith("%") else int(x))

In [23]:
mt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17722 entries, 0 to 1463
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Day           17722 non-null  object
 1   Hour          17722 non-null  object
 2   Condition     17722 non-null  object
 3   Temperature   17722 non-null  int64 
 4   Wind          17722 non-null  int64 
 5   Relative_hum  17722 non-null  int64 
 6   Pressure      17722 non-null  int64 
 7   gusts         17722 non-null  int64 
dtypes: int64(5), object(3)
memory usage: 1.2+ MB


## Convertimos a formato DateTime las columnas *"Day"* y *"date_time"* para poder tener posteriormente la tabla ordenada cronológicamente.

In [24]:
mt['Day'] = pd.to_datetime(mt['Day'], format='%A %d %B %Y').dt.strftime('%Y-%m-%d')

In [25]:
mt['date_time'] = pd.to_datetime(mt['Day'] + ' ' + mt['Hour'])

In [26]:
mt.head()

,Day,Hour,Condition,Temperature,Wind,Relative_hum,Pressure,gusts,date_time
0,2023-10-01,00:00,Clear,20,2,73,1025,0,2023-10-01 00:00:00
1,2023-10-01,00:30,Clear,19,3,78,1025,0,2023-10-01 00:30:00
2,2023-10-01,01:00,Clear,19,3,78,1025,0,2023-10-01 01:00:00
3,2023-10-01,01:30,Clear,19,3,73,1025,0,2023-10-01 01:30:00
4,2023-10-01,02:00,Clear,18,3,77,1025,0,2023-10-01 02:00:00


In [27]:
mt = mt.sort_values(by='date_time', ascending=False)
mt = mt.reset_index(drop = True)

In [28]:
mt[mt.duplicated()]

,Day,Hour,Condition,Temperature,Wind,Relative_hum,Pressure,gusts,date_time


## Añadimos un *id* a cada registro, reordenamos las columnas de la tabla y corregimos algunos de sus nombres.

In [29]:
mt['Metar_id'] = range(1, len(mt) + 1)

In [30]:
reorder = ["Metar_id","date_time","Day","Hour","Condition","Temperature","Wind","gusts","Relative_hum","Pressure"]
mt = mt[reorder]

In [31]:
mt = mt.rename(columns ={'gusts':'Gusts', 'date_time':'Date_time'})

In [32]:
mt.head()

,Metar_id,Date_time,Day,Hour,Condition,Temperature,Wind,Gusts,Relative_hum,Pressure
0,1,2023-10-31 23:30:00,2023-10-31,23:30,Fair,8,3,0,93,1017
1,2,2023-10-31 23:00:00,2023-10-31,23:00,Fair,8,1,0,87,1017
2,3,2023-10-31 22:30:00,2023-10-31,22:30,Fair,8,0,0,93,1017
3,4,2023-10-31 22:00:00,2023-10-31,22:00,Clear,8,0,0,93,1017
4,5,2023-10-31 21:30:00,2023-10-31,21:30,Clear,8,0,0,93,1017


## Exportamos los datos limpios en formato *csv* y *parquet*.

In [33]:
mt.to_csv("../metar/metars.csv", index=False)

In [34]:
mt.to_parquet('../metar/metar.gz', compression='gzip', index = False)